In [5]:
import os
import glob
import pandas as pd
import PyPDF2
from dotenv import load_dotenv, get_key
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from chromadb import PersistentClient, Settings
from langchain.vectorstores import Chroma
from langchain_voyageai import VoyageAIEmbeddings  # adjust the import as per your package

# Load environment variables from .env
load_dotenv()
VOYAGEAI_KEY = get_key(dotenv_path=".env", key_to_get="VOYAGEAI_KEY")

def load_documents_from_directory(directory):
    """
    Load documents from a directory. Supports TXT, CSV, and PDF files.
    Returns a list of Document objects with page_content and metadata.
    """
    documents = []
    for filepath in glob.glob(os.path.join(directory, "*")):
        ext = os.path.splitext(filepath)[1].lower()
        if ext == ".txt":
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()
            documents.append(Document(page_content=content, metadata={"source": filepath}))
        elif ext == ".csv":
            # Read CSV with pandas and convert to CSV string
            df = pd.read_csv(filepath)
            content = df.to_csv(index=False)
            documents.append(Document(page_content=content, metadata={"source": filepath}))
        elif ext == ".pdf":
            content = ""
            with open(filepath, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                for page in reader.pages:
                    text = page.extract_text()
                    if text:
                        content += text + "\n"
            documents.append(Document(page_content=content, metadata={"source": filepath}))
        # Extend for other file types if needed.
    return documents

# Load all documents from the specified directory
docs_directory = "./rag_docs"
documents = load_documents_from_directory(docs_directory)

# Initialize a RecursiveCharacterTextSplitter for chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Adjust chunk size as needed
    chunk_overlap=200     # Overlap helps maintain context between chunks
)

# Loop through each document and split it into chunks
combined_chunks = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        # Each chunk is stored as a Document with the original metadata
        combined_chunks.append(Document(page_content=chunk, metadata=doc.metadata))

# Set up the persistent Chroma client
DEFAULT_TENANT = "default_tenant"
DEFAULT_DATABASE = "default_database"
new_client = PersistentClient(
    path="./chroma_db",
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
    settings=Settings()
)

# Initialize VoyageAIEmbeddings with the API key and desired model
embeddings = VoyageAIEmbeddings(
    voyage_api_key=VOYAGEAI_KEY,
    model="voyage-large-2-instruct"
)

# Create a Chroma vector store from the chunked documents
vectorstore = Chroma.from_documents(
    documents=combined_chunks,
    embedding=embeddings,
    collection_name="umich_wn2025",
    client=new_client
)

print("Vector store created successfully with", len(combined_chunks), "chunks.")

/Users/satyamgoyal/anaconda3/envs/hermes/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector store created successfully with 6393 chunks.


In [5]:
from interaction import get_context_from_vector_db

user_input = "computer science"
context = get_context_from_vector_db(user_input)

Initialized Chroma vector store.


In [6]:
context

'Winter 2025,Regular Academic Session,Engineering,33203,Electrical Engineering And Computer Science (EECS), 370,5,Intro Computer Org,LEC,S R ,,,,,,,,01/08/2025,04/22/2025,ARR,ARR,"Manerkar, Brehob",4.00,\nWinter 2025,Regular Academic Session,Engineering,16029,Electrical Engineering And Computer Science (EECS), 370,11,Intro Computer Org,LAB,P R ,,,,,F,,,01/08/2025,04/22/2025,130-330PM,104 EWRE,Barstys,4.00,\nWinter 2025,Regular Academic Session,Engineering,10655,Electrical Engineering And Computer Science (EECS), 370,12,Intro Computer Org,LAB,P RW,M,,,,,,,01/08/2025,04/22/2025,430-630PM,138 NAME,Chlachidze,4.00,\nWinter 2025,Regular Academic Session,Engineering,10656,Electrical Engineering And Computer Science (EECS), 370,13,Intro Computer Org,LAB,P RW,,,,,F,,,01/08/2025,04/22/2025,1030-1230PM,1003 EECS,Vibhatasilpin,4.00,\nWinter 2025,Regular Academic Session,Engineering,30090,Computer Science and Engineering (CSE), 583,1,Advanced Compilers,LEC,P  W,,,W,,,,,01/08/2025,04/22/2025,130-23